In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report, roc_auc_score
from xgboost import XGBClassifier

# Load and sort data
df = pd.read_csv('/Users/cyruskurd/Documents/grad_programming/AML/Project work/combined_data_with_y.csv')
df['timestamp'] = pd.to_datetime(df['timestamp'])
df = df.sort_values('timestamp')

# Define maxout parameters
num_days_maxout = 3
threshold = 1.095 ** num_days_maxout

# Feature Engineering
df['SMA_5'] = df['close'].rolling(window=5).mean()
df['SMA_10'] = df['close'].rolling(window=10).mean()
df['SMA_20'] = df['close'].rolling(window=20).mean()
df['Bollinger_Upper'] = df['SMA_20'] + (df['close'].rolling(window=20).std() * 2)
df['Bollinger_Lower'] = df['SMA_20'] - (df['close'].rolling(window=20).std() * 2)
df['EMA_10'] = df['close'].ewm(span=10, adjust=False).mean()

# RSI Calculation
delta = df['close'].diff(1)
gain = delta.where(delta > 0, 0)
loss = -delta.where(delta < 0, 0)
avg_gain = gain.rolling(window=14).mean()
avg_loss = loss.rolling(window=14).mean()
rs = avg_gain / avg_loss
df['RSI'] = 100 - (100 / (1 + rs))

# Additional Indicators
df['ATR'] = df['close'].rolling(window=14).std()
df['Volume_SMA_10'] = df['vol'].rolling(window=10).mean()
df['Volume_Spike'] = (df['vol'] > df['Volume_SMA_10']).astype(int)
df['Rolling_Std_20'] = df['close'].rolling(window=20).std()

# Drop rows with NaN values
df.dropna(inplace=True)

# Define features and target
X = df[['SMA_5', 'SMA_10', 'SMA_20', 'EMA_10', 'Bollinger_Upper', 'Bollinger_Lower', 'RSI', 'ATR', 'Volume_Spike', 'Rolling_Std_20']]
y = df['y']

# Feature Scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train-test-validation split
split_date = '2020-01-01'

# Convert y to binary labels based on a threshold
binary_threshold = 0.1
y_binary = (y >= binary_threshold).astype(int)

# Redefine the train-test split with the binary labels
X_train_full, X_test, y_train_full, y_test = train_test_split(
    X_scaled[df['timestamp'] < split_date],
    y_binary[df['timestamp'] < split_date],
    test_size=0.2, random_state=42
)

X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size=0.25, random_state=42)

# Handle class imbalance using scale_pos_weight
negative_counts = (y_train == 0).sum()
positive_counts = (y_train == 1).sum()
scale_pos_weight = negative_counts / positive_counts

# Train XGBoost classifier
xgb_model = XGBClassifier(scale_pos_weight=scale_pos_weight, random_state=42, n_estimators=100,
                          max_depth=5, learning_rate=0.1, subsample=0.8)

xgb_model.fit(X_train, y_train)

# Predictions
y_pred_proba = xgb_model.predict_proba(X_test)[:, 1]
threshold = 0.5
y_pred = (y_pred_proba >= threshold).astype(int)

# Evaluation
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)
conf_matrix = confusion_matrix(y_test, y_pred)

print(f"XGBoost Accuracy: {accuracy:.4f}")
print(f"XGBoost F1 Score: {f1:.4f}")
print(f"ROC AUC Score: {roc_auc:.4f}")
print("Confusion Matrix:\n", conf_matrix)
print(classification_report(y_test, y_pred))

XGBoost Accuracy: 0.5482
XGBoost F1 Score: 0.0796
ROC AUC Score: 0.6380
Confusion Matrix:
 [[1025130  855388]
 [  20663   37875]]
              precision    recall  f1-score   support

           0       0.98      0.55      0.70   1880518
           1       0.04      0.65      0.08     58538

    accuracy                           0.55   1939056
   macro avg       0.51      0.60      0.39   1939056
weighted avg       0.95      0.55      0.68   1939056



In [2]:
scale_pos_weight

32.27099323961062

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report, roc_auc_score
from xgboost import XGBClassifier

# Load and sort data
df = pd.read_csv('/Users/cyruskurd/Documents/grad_programming/AML/Project work/combined_data_with_y.csv')
df['timestamp'] = pd.to_datetime(df['timestamp'])
df = df.sort_values('timestamp')

# Define maxout parameters
num_days_maxout = 3
threshold = 1.095 ** num_days_maxout

# Feature Engineering
df['SMA_5'] = df['close'].rolling(window=5).mean()
df['SMA_10'] = df['close'].rolling(window=10).mean()
df['SMA_20'] = df['close'].rolling(window=20).mean()
df['Bollinger_Upper'] = df['SMA_20'] + (df['close'].rolling(window=20).std() * 2)
df['Bollinger_Lower'] = df['SMA_20'] - (df['close'].rolling(window=20).std() * 2)
df['EMA_10'] = df['close'].ewm(span=10, adjust=False).mean()

# RSI Calculation
delta = df['close'].diff(1)
gain = delta.where(delta > 0, 0)
loss = -delta.where(delta < 0, 0)
avg_gain = gain.rolling(window=14).mean()
avg_loss = loss.rolling(window=14).mean()
rs = avg_gain / avg_loss
df['RSI'] = 100 - (100 / (1 + rs))

# Additional Indicators
df['ATR'] = df['close'].rolling(window=14).std()
df['Volume_SMA_10'] = df['vol'].rolling(window=10).mean()
df['Volume_Spike'] = (df['vol'] > df['Volume_SMA_10']).astype(int)
df['Rolling_Std_20'] = df['close'].rolling(window=20).std()

# Drop rows with NaN values
df.dropna(inplace=True)

# Define features and target
X = df[['SMA_5', 'SMA_10', 'SMA_20', 'EMA_10', 'Bollinger_Upper', 'Bollinger_Lower', 'RSI', 'ATR', 'Volume_Spike', 'Rolling_Std_20']]
y = df['y']

# Feature Scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train-test-validation split
split_date = '2020-01-01'

# Convert y to binary labels based on a threshold
binary_threshold = 0.1
y_binary = (y >= binary_threshold).astype(int)

# Redefine the train-test split with the binary labels
X_train_full, X_test, y_train_full, y_test = train_test_split(
    X_scaled[df['timestamp'] < split_date],
    y_binary[df['timestamp'] < split_date],
    test_size=0.2, random_state=42
)

X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size=0.25, random_state=42)

# Handle class imbalance using scale_pos_weight
negative_counts = (y_train == 0).sum()
positive_counts = (y_train == 1).sum()
scale_pos_weight = negative_counts / positive_counts

# Train XGBoost classifier
xgb_model = XGBClassifier(scale_pos_weight=1, random_state=42, n_estimators=100,
                          max_depth=5, learning_rate=0.1, subsample=0.8)

xgb_model.fit(X_train, y_train)

# Predictions
y_pred_proba = xgb_model.predict_proba(X_test)[:, 1]
threshold = 0.5
y_pred = (y_pred_proba >= threshold).astype(int)

# Evaluation
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)
conf_matrix = confusion_matrix(y_test, y_pred)

print(f"XGBoost Accuracy: {accuracy:.4f}")
print(f"XGBoost F1 Score: {f1:.4f}")
print(f"ROC AUC Score: {roc_auc:.4f}")
print("Confusion Matrix:\n", conf_matrix)
print(classification_report(y_test, y_pred))

XGBoost Accuracy: 0.9698
XGBoost F1 Score: 0.0000
ROC AUC Score: 0.6373
Confusion Matrix:
 [[1880518       0]
 [  58538       0]]


/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


              precision    recall  f1-score   support

           0       0.97      1.00      0.98   1880518
           1       0.00      0.00      0.00     58538

    accuracy                           0.97   1939056
   macro avg       0.48      0.50      0.49   1939056
weighted avg       0.94      0.97      0.95   1939056



/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
